# A biblioteca requests
A primeira coisa que precisamos fazer para realizar o web scrap é baixar a página. Podemos baixar as páginas utilizando a biblioteca requests do Python. A biblioteca requests fará uma solicitação GET ao servidor, que fará o download dos conteúdos HTML da página solicitada para nós.

In [1]:
import requests

page = requests.get("http://dataquestio.github.io/web-scraping-pages/simple.html")
page

<Response [200]>

Após executar nossa solicitação, nós receberemos um objeto Response. Esse objeto tem uma propriedade status_code que indica se a página foi baixada com sucesso.

In [2]:
page.status_code #Um status_code com resultado 200 significa que a página foi baixada com sucesso. 


200

Nós podemos exibir o conteúdo HTML de uma página utilizando a propriedade content.


In [3]:
page.content

b'<!DOCTYPE html>\n<html>\n    <head>\n        <title>A simple example page</title>\n    </head>\n    <body>\n        <p>Here is some simple content for this page.</p>\n    </body>\n</html>'

# Analisando uma página com BeautifulSoup
Como você pode ver acima, agora nós baixamos um documento HTML.

Nós podemos utilizar a biblioteca BeautifulSoup para analisar esse documento e extrair o texto da tag p. Primeiro, nós temos que importar a biblioteca e criar uma instância da classe BeautifulSoup para analisar o documento.

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

Agora, nós podemos exibir o conteúdo HTML da página, formatado corretamente, utilizando o método prettify no objeto BeautifulSoup.

In [5]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   A simple example page
  </title>
 </head>
 <body>
  <p>
   Here is some simple content for this page.
  </p>
 </body>
</html>


Como todas as tags estão aninhadas, podemos nos movimentar pela estrutura um nível por vez. Podemos selecionar primeiro os elementos dos níveis superiores da página utilizando a propriedade children do soup. Note que children retorna um gerador de listas, para que possamos chamar sua função list.



In [6]:
list(soup.children)

['html', '\n', <html>
 <head>
 <title>A simple example page</title>
 </head>
 <body>
 <p>Here is some simple content for this page.</p>
 </body>
 </html>]

Acima, vemos que existem duas tags no nível superior da página – a tag inicial <!DOCTYPE html> e a tag <html>. Existe também um caractere de nova linha (\n) na lista. Vamos ver o tipo de cada elemento da lista:

In [6]:
[type(item) for item in list(soup.children)]

[bs4.element.Doctype, bs4.element.NavigableString, bs4.element.Tag]

Como você pode ver, todos os itens são objetos BeautiSoup. O primeiro é o objeto Doctype, que contém informações sobre o tipo de documento. O segundo é o NavigableString, que representa o texto encontrado no documento HTML. O item final é um objeto Tag, que contém outras tags aninhadas. O tipo de objeto mais importante, e com o qual nós vamos trabalhar com mais frequência, é o objeto Tag. O objeto Tag nos permite navegar pelo documento HTML e extrair outras tags e textos.

Agora nós podemos selecionar a tag html e suas filhas pelo terceiro item da lista:

In [7]:
html = list(soup.children)[2]

Cada item da lista retornado pelo objeto children também é um objeto de BeautifulSoup, então nós também podemos chamar o método children no html.

Agora, nós podemos encontrar as filhas dentro da tag html:

In [8]:
list(html.children)

['\n', <head>
 <title>A simple example page</title>
 </head>, '\n', <body>
 <p>Here is some simple content for this page.</p>
 </body>, '\n']

Como você pode ver acima, existem duas tags aqui, head e body. Nós queremos extrair o texto dentro da tag p, então nós vamos mais afundo em body.

In [10]:
body = list(html.children)[3]

Agora, nós podemos pegar a tag p encontrando as filhas da tag body:

In [11]:
list(body.children)

['\n', <p>Here is some simple content for this page.</p>, '\n']

Nós podemos isolar a tag p:

In [12]:
p = list(body.children)[1]

Assim que isolarmos a tag, nós podemos utilizar o método get_text para extrair todo o texto dentro dela

In [13]:
list(body.children)

['\n', <p>Here is some simple content for this page.</p>, '\n']

# Encontrando todas as instâncias de uma tag de uma vez
O que fizemos acima foi útil para aprendermos como navegar pela página, mas precisamos de muitos comandos para algo simples. Se nós queremos extrair uma única tag, nós podemos utilizar o método find_all, que encontrará todas as instâncias de uma tag na página.

In [14]:
soup = BeautifulSoup(page.content, 'html.parser')
soup.find_all('p')

[<p>Here is some simple content for this page.</p>]

Note que find_all retorna uma lista, então vamos ter que procurar, ou utilizar indexação de lista, para extrair o texto.

In [15]:
soup.find_all('p')[0].get_text()

'Here is some simple content for this page.'

Se você quiser, porém, encontrar somente a primeira instância de uma tag, você pode utilizar o método find, que retorna um único objeto BeautifulSoup.

In [16]:
soup.find('p')

<p>Here is some simple content for this page.</p>

# Buscando tags por classe e id
Nós apresentamos classes e ids acima, mas provavelmente não ficou claro por que elas são úteis. Classes e ids são utilizadas pelo CSS para determinar para quais elementos HTML certos estilos são aplicados. Nós também podemos utilizá-las enquanto fazemos o scrap para especificar os elementos em que queremos fazer scrap. Para ilustrar esse princípio, vamos trabalhar com a seguinte página:

In [9]:
page = requests.get("http://dataquestio.github.io/web-scraping-pages/ids_and_classes.html")
soup = BeautifulSoup(page.content, 'html.parser')
soup

<html>
<head>
<title>A simple example page</title>
</head>
<body>
<div>
<p class="inner-text first-item" id="first">
                First paragraph.
            </p>
<p class="inner-text">
                Second paragraph.
            </p>
</div>
<p class="outer-text first-item" id="second">
<b>
                First outer paragraph.
            </b>
</p>
<p class="outer-text">
<b>
                Second outer paragraph.
            </b>
</p>
</body>
</html>

Agora nós podemos utilizar o método find_all para buscar por todos os itens por classe e id. No exemplo abaixo, vamos procurar por qualquer tag p que tenha a classe outer-text:

In [18]:
soup.find_all('p', class_='outer-text')

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>, <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

No exemplo abaixo, estamos procurando por qualquer tag que tenha a classe outer-text:

In [20]:
soup.find_all(class_="outer-text")

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>, <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

Nós também podemos buscar elementos pela id.

In [21]:
soup.find_all(id="first")

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>]

# Utilizando seletores CSS
Você também pode realizar buscas por itens utilizando os seletores CSS. Esses seletores são como a linguagem CSS permite que os desenvolvedores especifiquem as tags HTML que receberão os estilos. Aqui estão alguns exemplos:

- p a: Encontra todas as tags a dentro de uma tag p; 
- body p a: Encontra as tags a dentro de uma tag p dentro da tag body;
- html body: Encontra todas as tags body dentro de uma tag html;
- p.outer-text: Encontra todas as tags p com uma classe outer-text;
- p#first: Encontra todas as tags p com um id first;
- body p.outer-text: Encontra quaisquer tags p com uma classe outer-text dentro de uma tag body.

Você pode aprender mais sobre os seletores CSS https://developer.mozilla.org/en-US/docs/Web/Guide/CSS/Getting_started/Selectors

Os objetos BeautifulSoup suportam a busca em páginas utilizando os seletores CSS através do método select. Nós podemos utilizar os seletores CSS para encontrar todas as tags p de nossa página que estiverem dentro de uma div desta maneira:

In [22]:
soup.select("div p") #método select retorna uma lista de objetos BeautifulSoup, assim como find e find_all.

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>, <p class="inner-text">
                 Second paragraph.
             </p>]